# Chiral solutions for Type-II Dirac seesaw with a Dark Symmetry

In [ ]:
import pandas as pd
import numpy as np
import itertools

### Load full solutions

In [ ]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

Number of solutions with repeated charges: $(\nu,\nu,\cdots)$

In [ ]:
ds

,l,k,solution,gcd,n
0,"[-1, 0, -1, 0, -1]","[-1, 0, -1, 1, -1]","[1, 4, 4, 4, -5, -5, -5, -5, -5, 6, 6]",1,11
1,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7
2,"[1, -1, 0, 1, -1]","[-2, 0, -1, -2, 0]","[1, 1, -2, -2, -3, -3, 5, 5, 5, 5, -6, -6]",2,12
3,"[-1, 1, -1, 1, -1]","[-2, -1, 0, -1, 0]","[1, 1, 1, 1, 2, 2, 2, -3, -4, -4, -6, 7]",1,12
4,"[1, 0, 1, 0, -1]","[-2, -1, -2, -1, 0]","[1, 1, 1, -2, -2, -5, -5, 6, 6, 7, -8]",1,11
...,...,...,...,...,...
390069,"[-3, -15, -6]","[-3, 8, 9]","[8, -9, -9, 11, 25, -29, -29, 32]",36,8
390070,"[-10, -5, -15]","[-10, -12, 12]","[5, 5, 5, -17, -27, -27, 28, 28]",100,8
390071,"[15, -1, 15]","[-14, -13, -14]","[14, -15, -16, 17, -30, 31, 31, -32]",1,8
390072,"[-15, -9, -1]","[-13, -14, -12]","[11, -13, -15, 19, 23, -26, -30, 31]",18,8


In [ ]:
dsp=ds[ds['solution'].apply(lambda l: len(l)-len(set(l))>0)].reset_index(drop=True)
dsp.shape

(280959, 5)

Some reordering

In [ ]:
dsp['nmax']=dsp['solution'].apply(lambda l: map(abs,l)).apply(max)
dsp=dsp.sort_values(['n','nmax']).reset_index(drop=True)
#Show the first and last one solutions
dsp.iloc[[0,1,2,-1]]

,l,k,solution,gcd,n,nmax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6
2,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,11
280958,"[-1, -2, 3, -3, 2]","[-1, 3, 2, 0, -2]","[1, 6, 6, 8, -9, -9, 21, 21, -25, -26, -26, 32]",4,12,32


## 1. Check phenomenological conditions
$$(\nu,\nu,\nu,\psi_1,\psi_2,\cdots)$$


$$ \nu+s=0$$

$$ \psi_i+\psi_j=|s|$$

Test:

In [ ]:
s=-1
ms=set([x for x in itertools.permutations([-4, -4, 5],2) if x[0]<=x[1]])
xs=[ x for x in ms if abs(x[0]+x[1])==abs(s) ]
[x for sublist in xs for x in sublist]

[-4, 5]

Real implementation

In [ ]:
def conditions(l):
    #Fix first set of repeated integers as 𝜈
    νs=list(set([x for x in l if l.count(x)>=2 ]))
    ν=νs[0]
    #implement the phenomenological condition:
    s=-ν
    #Remove neutrinos from list, `l` and get extra quiral fermions
    ψs=[x for x in l if x!=ν]
    if len(ψs)<=2:
        ms=ψs
    else:
        ms=set([x for x in itertools.permutations(ψs,2) if x[0]<=x[1] ])
    xs=[ x for x in ms if abs(x[0]+x[1])==abs(s) ]
    #flatten list of list into list
    massive=[x for sublist in xs for x in sublist]

    return list( set(ψs).difference(set(massive)) )

In [ ]:
dsp['massless']=dsp['solution'].apply(conditions)

In [ ]:
dsp

,l,k,solution,gcd,n,nmax,massless
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5,[]
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6,[]
2,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,11,"[11, 5, -9]"
3,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,11,"[1, -4, -11]"
4,"[-2, -1]","[-2, -3]","[2, 2, 3, -8, -12, 13]",1,6,13,"[-8, 3, -12, 13]"
...,...,...,...,...,...,...,...
280954,"[-2, -3, -1, 3, 2]","[-1, 3, 1, -1, -3]","[1, 2, -10, 12, 19, 19, -23, -24, -30, -30, 32...",2,12,32,"[1, 2, -30, -24, -23, 12, 19, -10]"
280955,"[-1, -2, 0, 3, 2]","[-1, 3, 1, -1, -2]","[1, 1, 1, 1, 8, -12, -19, 20, 21, -23, -31, 32]",1,12,32,"[8, -23, -12, 21]"
280956,"[-1, 2, -3, -2, -3]","[-1, 3, 2, -2, -1]","[1, 2, 2, 4, 5, -7, -7, -9, -11, 25, 27, -32]",2,12,32,"[1, 27]"
280957,"[-1, 3, -2, 2, -2]","[-1, 3, 2, -1, -2]","[1, 3, 4, 8, -9, -9, -12, 19, 19, -28, -28, 32]",1,12,32,"[32, 1, 3, 4, -28, 8, -12, -9]"


In [ ]:
sltns=dsp[dsp['massless'].apply(len)<=2].reset_index(drop=True)

In [ ]:
sltns.shape

(12584, 7)

In [ ]:
sltns[:10]

,l,k,solution,gcd,n,nmax,massless
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,5,[]
1,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,6,[]
2,"[1, 0]","[-1, -5, 0]","[2, -3, -10, 13, 13, -15]",2,6,15,[]
3,"[0, 2]","[-1, -2, 1]","[1, 1, 8, -11, -16, 17]",1,6,17,"[8, -11]"
4,"[1, 2]","[-1, -2, 0]","[8, -11, -11, 16, 17, -19]",1,6,19,"[16, 17]"
5,"[1, 0]","[-1, -4, 0]","[3, -5, -12, 17, 17, -20]",1,6,20,[]
6,"[1, -2]","[-1, -4, 0]","[10, -13, -13, 17, 22, -23]",1,6,23,"[17, 22]"
7,"[1, 0]","[-1, -7, 0]","[3, -4, -21, 25, 25, -28]",2,6,28,[]
8,"[1, 2]","[-1, 2, 0]","[4, -11, -11, 20, 29, -31]",1,6,31,"[4, 29]"
9,"[0, 3]","[-1, 2, 1]","[8, -13, -13, 19, 31, -32]",2,6,32,"[8, 31]"
